In [30]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import numpy as np
import re

In [13]:
# load data food
df_food = pd.read_excel("food.xlsx")
df_food

,(A1) CITY,(A2) PROVINCE,(B3) GENDER,(B4) AGE,(B5) RELIGION,(B6) ETHNIC,(B7) EDUCATION LEVEL,(B8) LIVED IN URBAN AREA (YEARS),(B9) OCCUPATION,(B10) SOCIAL CLAS,...,(E19) RELIGIOUS VALUE IN EATING PLACES,(E20) RELIGIOUS VALUE IN CONSUMPTION PATTERN,(E21) CUSTOM VALUE IN TYPE OF FOOD,(E22) CUSTOM VALUE IN EATING PLACES,(E23) CUSTOM VALUE IN CONSUMPTION PATTERN,(24) PRACTICE OF FOOD CONSUMPTION BASED ON RELIGIOUS RULES,(25) PRACTICE OF FOOD CONSUMPTION BASED ON CUSTOM RULES,(E26) BUYING FOOD PRODUCT IN THE RIGHT PLACE BASED ON RELIGIOUS RULES,(E27 TYPE OF DISHES VARIATION BASED ON RELIGIOUS RULES,(E28) PRACTICE OF CONSUMING TRADITIONAL FOOD
0,Jakarta,DKI Jakarta,Female,31,Islam,Java,Senior High School,31,Private Employees,Lower Class,...,Neither agree nor disagree,Agree,Strongly Agree,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree,Disagree,Neither agree nor disagree,Agree,Agree
1,Jakarta,DKI Jakarta,Female,36,Islam,Java,Bachelor degree,32,Private Employees,Lower Class,...,Agree,Agree,Strongly Agree,Agree,Agree,Disagree,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree,Strongly Agree
2,Jakarta,DKI Jakarta,Male,24,Islam,Java,Senior High School,24,Private Employees,Middle Class,...,Agree,Neither agree nor disagree,Neither agree nor disagree,Disagree,Agree,Disagree,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree
3,Jakarta,DKI Jakarta,Female,63,Islam,Other,Senior High School,32,Private Employees,Lower Class,...,Agree,Neither agree nor disagree,Agree,Neither agree nor disagree,Agree,Agree,Neither agree nor disagree,Disagree,Neither agree nor disagree,Agree
4,Jakarta,DKI Jakarta,Male,43,Protestant,Batak,Primary School,43,Government Employees,Lower Class,...,Strongly Agree,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree,Strongly Agree,Strongly Agree,Agree,Disagree,Agree,Agree
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
705,Denpasar,Bali,Female,64,Islam,Java,Junior High School,24,Government Employees,Middle Class,...,Strongly Agree,Neither agree nor disagree,Agree,Strongly Agree,Strongly Agree,Strongly Agree,Neither agree nor disagree,Agree,Neither agree nor disagree,Neither agree nor disagree
706,Denpasar,Bali,Male,51,Protestant,Java,Master and doctoral degree,24,Enterpreneur,Upper Class,...,Neither agree nor disagree,Agree,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree
707,Denpasar,Bali,Male,26,Hindu,Batak,Senior High School,26,Enterpreneur,Lower Class,...,Agree,Strongly Agree,Strongly Agree,Agree,Agree,Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree
708,Denpasar,Bali,Male,30,Hindu,Bali,Senior High School,30,Enterpreneur,Upper Class,...,Neither agree nor disagree,Strongly Agree,Agree,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree,Strongly Agree,Agree,Strongly Agree,Strongly Agree


In [6]:
#info dataset
df_food.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 710 entries, 0 to 709
Data columns (total 28 columns):
 #   Column                                                                 Non-Null Count  Dtype 
---  ------                                                                 --------------  ----- 
 0   (A1) CITY                                                              710 non-null    object
 1   (A2) PROVINCE                                                          710 non-null    object
 2   (B3) GENDER                                                            710 non-null    object
 3   (B4) AGE                                                               710 non-null    int64 
 4   (B5) RELIGION                                                          710 non-null    object
 5   (B6) ETHNIC                                                            710 non-null    object
 6   (B7) EDUCATION LEVEL                                                   710 non-null    object
 7  

## Cleansing Data

In [38]:
#change columns name
df_food.rename(columns=lambda x:re.sub(r'\([^)]*\)','',x).strip())

#change column
df_food.rename(columns={'(E27 TYPE OF DISHES VARIATION BASED ON RELIGIOUS RULES':'TYPE OF DISHES VARIATION BASED ON RELIGIOUS RULES'})

print(df_food.columns)

Index(['CITY', 'PROVINCE', 'GENDER', 'AGE', 'RELIGION', 'ETHNIC',
       'EDUCATION LEVEL', 'LIVED IN URBAN AREA', 'OCCUPATION', 'SOCIAL CLAS',
       'HOUSEHOLD MEMBER', 'HOUSEHOLD INCOME', 'TIME OF INCOME RECEIPT',
       'HOUSEHOLD EXPENDITURE', '% MONTHLY EXPENDITURE FOR FOOD',
       'HEALTH FACTOR', 'PRICE FACTOR)', 'RELIGIOUS VALUE IN TYPE OF FOOD',
       'RELIGIOUS VALUE IN EATING PLACES',
       'RELIGIOUS VALUE IN CONSUMPTION PATTERN',
       'CUSTOM VALUE IN TYPE OF FOOD', 'CUSTOM VALUE IN EATING PLACES',
       'CUSTOM VALUE IN CONSUMPTION PATTERN',
       'PRACTICE OF FOOD CONSUMPTION BASED ON RELIGIOUS RULES',
       'PRACTICE OF FOOD CONSUMPTION BASED ON CUSTOM RULES',
       'BUYING FOOD PRODUCT IN THE RIGHT PLACE BASED ON RELIGIOUS RULES',
       'TYPE OF DISHES VARIATION BASED ON RELIGIOUS RULES',
       'PRACTICE OF CONSUMING TRADITIONAL FOOD'],
      dtype='object')


In [16]:
#cek missing value food
#cek nilai null pada data train
#menghitung jumlah nilai NaN pada setiap kolom data frame df_train
null_valu = df_food.isna().sum().sort_values(ascending=False)

#menghitung persentase nilai NaN pada setiap kolom data frame df_train
pres_Nan = round(null_valu/len(df_food)*100,2)

#menyusun hasil perhitungan persentase nilai NaN pada setiap kolom ke dalam suatu data frame
#dengan kolom 'NaN %' dan menampilkan 5 baris pertama
pd.DataFrame({'NaN %':pres_Nan}).head()


,NaN %
(A1) CITY,0.0
(A2) PROVINCE,0.0
(E27 TYPE OF DISHES VARIATION BASED ON RELIGIOUS RULES,0.0
(E26) BUYING FOOD PRODUCT IN THE RIGHT PLACE BASED ON RELIGIOUS RULES,0.0
(25) PRACTICE OF FOOD CONSUMPTION BASED ON CUSTOM RULES,0.0


In [14]:
#cek outlier
print(f'Jumlah baris sebelum memfilter outlier: {len(df_food)}')

filtered_entries = np.array([True] * len(df_food))
for col in ['(B4) AGE','(B8) LIVED IN URBAN AREA (YEARS)','(B11) HOUSEHOLD MEMBER',
           '(C15) % MONTHLY EXPENDITURE FOR FOOD']:
    Q1 = df_food[col].quantile(0.25)
    Q3 = df_food[col].quantile(0.75)
    IQR = Q3 - Q1
    low_limit = Q1 - (IQR * 1.5)
    high_limit = Q3 + (IQR * 1.5)

    filtered_entries = ((df_food[col] >= low_limit) & (df_food[col] <= high_limit)) & filtered_entries
    
df_food = df_food[filtered_entries]

print(f'Jumlah baris setelah memfilter outlier: {len(df_food)}')

Jumlah baris sebelum memfilter outlier: 710
Jumlah baris setelah memfilter outlier: 419


In [23]:
#cek duplikasi data
df_food.duplicated(subset=['(A1) CITY','(A1) CITY','(A1) CITY','(A1) CITY','(B4) AGE','(B8) LIVED IN URBAN AREA (YEARS)','(B11) HOUSEHOLD MEMBER',
           '(C15) % MONTHLY EXPENDITURE FOR FOOD','(E23) CUSTOM VALUE IN CONSUMPTION PATTERN','(B11) HOUSEHOLD MEMBER',
                          '(B11) HOUSEHOLD MEMBER']).sum()

3

In [26]:
print(df_food.duplicated().sum())
df_food.drop_duplicates()
print(df_food.duplicated().sum())

0
0


In [27]:
#describe data numerik food
df_food.describe()

,(B4) AGE,(B8) LIVED IN URBAN AREA (YEARS),(B11) HOUSEHOLD MEMBER,(C15) % MONTHLY EXPENDITURE FOR FOOD
count,419.000000,419.000000,419.0,419.000000
mean,40.961814,27.976134,6.0,41.947494
std,11.965400,7.973905,0.0,17.397055
min,22.000000,7.000000,6.0,18.000000
25%,31.000000,23.000000,6.0,26.000000
50%,38.000000,28.000000,6.0,38.000000
75%,52.000000,33.000000,6.0,58.000000
max,64.000000,50.000000,6.0,76.000000


In [29]:
#describe data object
df_food.describe(include='O')

,(A1) CITY,(A2) PROVINCE,(B3) GENDER,(B5) RELIGION,(B6) ETHNIC,(B7) EDUCATION LEVEL,(B9) OCCUPATION,(B10) SOCIAL CLAS,(C12) HOUSEHOLD INCOME (MONTHLY IN MILLION IDR),(C13) TIME OF INCOME RECEIPT,...,(E19) RELIGIOUS VALUE IN EATING PLACES,(E20) RELIGIOUS VALUE IN CONSUMPTION PATTERN,(E21) CUSTOM VALUE IN TYPE OF FOOD,(E22) CUSTOM VALUE IN EATING PLACES,(E23) CUSTOM VALUE IN CONSUMPTION PATTERN,(24) PRACTICE OF FOOD CONSUMPTION BASED ON RELIGIOUS RULES,(25) PRACTICE OF FOOD CONSUMPTION BASED ON CUSTOM RULES,(E26) BUYING FOOD PRODUCT IN THE RIGHT PLACE BASED ON RELIGIOUS RULES,(E27 TYPE OF DISHES VARIATION BASED ON RELIGIOUS RULES,(E28) PRACTICE OF CONSUMING TRADITIONAL FOOD
count,419,419,419,419,419,419,419,419,419,419,...,419,419,419,419,419,419,419,419,419,419
unique,5,5,2,5,8,6,6,3,38,3,...,4,4,4,4,4,4,4,4,4,4
top,Denpasar,Bali,Female,Islam,Java,Diploma,Enterpreneur,Lower Class,03:04:00,Daily,...,Strongly Agree,Strongly Agree,Agree,Neither agree nor disagree,Neither agree nor disagree,Agree,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree
freq,99,99,246,254,177,153,120,145,32,143,...,165,202,167,203,199,177,220,184,220,213


### Analisa data

Dari data yang sudah dideskripsikan, berikut beberapa informasi dasar yang dapat diambil:
* umur orang yang ada pada survey rata-rata berumur 40 tahun dengan rata-rata lama tinggal di daerah urban selama 27 tahun dan presentase biaya bulanan yang dihabiskan untuk makanan bulanan dalam persen yaitu rata-rata 41%
* daerah pengisi survey paling banyak dari kota denpasar, provinsi bali
* gender pengisi survey didominasi oleh perempuan dengan total 246 pengisi dari 419
* jenjang pengisi survey paling banyak dari tingkat diploma

berikut visualisasi variabel yang ada untuk dilihat keterkaitan antar data: